# Model Building with First Order Logic

Kevin Nolasco

Cabrini University

MCIS565 - Natural Language Processing

05/01/2022

## Prompt

Select three or four contiguous sentences from a book for children. A possible source of examples are the collections of stories in nltk.corpus.gutenberg: bryant-stories.txt, burgess-busterbrown.txt and edgeworth-parents.txt. Develop a grammar which will allow your sentences to be translated into first order logic, and build a model which will allow those translations to be checked for truth or falsity.

## Load Data

We will use nltk.corpus.gutenberg to load our sentences. Then we will peak at the first 20 sentences to find 3 that would work well for this assignment.

In [15]:
import nltk
all_sents = nltk.corpus.gutenberg.sents('bryant-stories.txt')

In [16]:
print(len(all_sents))
for ind ,sents in enumerate(all_sents[:20]):
    print(ind, sents)

2863
0 ['[', 'Stories', 'to', 'Tell', 'to', 'Children', 'by', 'Sara', 'Cone', 'Bryant', '1918', ']']
1 ['TWO', 'LITTLE', 'RIDDLES', 'IN', 'RHYME']
2 ['There', "'", 's', 'a', 'garden', 'that', 'I', 'ken', ',', 'Full', 'of', 'little', 'gentlemen', ';', 'Little', 'caps', 'of', 'blue', 'they', 'wear', ',', 'And', 'green', 'ribbons', ',', 'very', 'fair', '.']
3 ['(', 'Flax', '.)']
4 ['From', 'house', 'to', 'house', 'he', 'goes', ',', 'A', 'messenger', 'small', 'and', 'slight', ',', 'And', 'whether', 'it', 'rains', 'or', 'snows', ',', 'He', 'sleeps', 'outside', 'in', 'the', 'night', '.']
5 ['(', 'The', 'path', '.)']
6 ['THE', 'LITTLE', 'YELLOW', 'TULIP']
7 ['Once', 'there', 'was', 'a', 'little', 'yellow', 'Tulip', ',', 'and', 'she', 'lived', 'down', 'in', 'a', 'little', 'dark', 'house', 'under', 'the', 'ground', '.']
8 ['One', 'day', 'she', 'was', 'sitting', 'there', ',', 'all', 'by', 'herself', ',', 'and', 'it', 'was', 'very', 'still', '.']
9 ['Suddenly', ',', 'she', 'heard', 'a', 'little',

By looking at the first 20 sentences, we can see that sentences 7,8 and 9 are good sentences to use for this project.

In [17]:
sentences_to_use = all_sents[7:10]
for sent in sentences_to_use:
    print(sent)

['Once', 'there', 'was', 'a', 'little', 'yellow', 'Tulip', ',', 'and', 'she', 'lived', 'down', 'in', 'a', 'little', 'dark', 'house', 'under', 'the', 'ground', '.']
['One', 'day', 'she', 'was', 'sitting', 'there', ',', 'all', 'by', 'herself', ',', 'and', 'it', 'was', 'very', 'still', '.']
['Suddenly', ',', 'she', 'heard', 'a', 'little', '_tap', ',', 'tap', ',', 'tap_', ',', 'at', 'the', 'door', '.']


## Data Cleaning

We will remove punctuations and the underscores that are attached to "tap" before moving on to building the grammar.

In [18]:
# remove punctuations
cleaned_sentences = [[word for word in sent if word.isalpha()] for sent in sentences_to_use]
for sent in cleaned_sentences:
    print(sent)

['Once', 'there', 'was', 'a', 'little', 'yellow', 'Tulip', 'and', 'she', 'lived', 'down', 'in', 'a', 'little', 'dark', 'house', 'under', 'the', 'ground']
['One', 'day', 'she', 'was', 'sitting', 'there', 'all', 'by', 'herself', 'and', 'it', 'was', 'very', 'still']
['Suddenly', 'she', 'heard', 'a', 'little', 'tap', 'at', 'the', 'door']


### Sentence 1

Before creating the grammar for sentence 1, let's determine how we can simplify the information given in sentence 1.

In [19]:
sent1 = cleaned_sentences[0]
' '.join(sent1)

'Once there was a little yellow Tulip and she lived down in a little dark house under the ground'

We can summarize the information given in the sentence above as "The little yellow Tulip lives in a little dark house."

I want the logic for this sentence to be:

"There exists a Tulip that is little and yellow and there exists a house that is little and dark and the the Tulip lives in the house."

Using first-order logic, this could be:

$\left[\exists t.(Tulip(t) \& little(t) \& yellow(t)) \rightarrow \exists h.((House(h) \& little(h) \& dark(h)) \& lives(t, h)) \right]$

Which translates to "if (there exists a tulip that is little and yellow) then (there exists a house that is little and dark and the tulip lives in the house)"

Now let's see the POS for the words used in the summarized sentence.

In [20]:
simple_sent_1 = """
The little yellow Tulip lives in a little dark house
"""
simple_sent_1_tokenized = simple_sent_1.split()
for item in nltk.pos_tag(simple_sent_1_tokenized):
    print(item)


('The', 'DT')
('little', 'JJ')
('yellow', 'JJ')
('Tulip', 'NNP')
('lives', 'VBZ')
('in', 'IN')
('a', 'DT')
('little', 'JJ')
('dark', 'NN')
('house', 'NN')


In [21]:
# the grammar below is saved as a .fcfg file in the current directory
grammar = r"""
% start S
# Grammar Rules
S[SEM = <?subj(?vp)>] -> NP[SEM=?subj] VP[SEM=?vp]
NP[SEM=<?dt(?jj, ?jj, ?nnp)>] -> DT[SEM=?dt] JJ[SEM=?jj] JJ[SEM=?jj] NNP[SEM=?nnp]
NP[SEM=<?jj(?nn, ?nn)>] -> JJ[SEM=?jj] NN[SEM=?nn] NN[SEM=?nn]
VP[SEM=<?vbz(?pp, ?np)>] -> VBZ[SEM=<?vbz>] PP[SEM=?pp] NP[SEM=?np]
PP[SEM=?dt] -> IN DT[SEM=?dt]
# Lexical Rules
DT[SEM=<\P Q R S.(exists x.((R(x) & P(x) & Q(x)) -> (S)))>] -> 'The'
JJ[SEM=<\x.little(x)>] -> 'little'
JJ[SEM=<\x.yellow(x)>] -> 'yellow'
NNP[SEM=<\x.Tulip(x)>] -> 'Tulip'
VBZ[SEM=<\P.(P & lives(x, y))>] -> 'lives'
IN -> 'in'
DT[SEM=<\P Q R.(exists y.(R(y) & P(y) & Q(y)))>] -> 'a'
NN[SEM=<\x.dark(x)>] -> 'dark'
NN[SEM=<\x.House(x)>] -> 'house'
"""

In [33]:
parser = nltk.parse.load_parser('grammar1.fcfg', trace = 0)
for tree in parser.parse(simple_sent_1_tokenized):
    print(tree.label()['SEM'])

There should be some results here, but the parser is not parsing the sentence correctly and I am not sure how to fix this. Since the sentence can't be parsed correctly, I cannot complete the remaining parts of the assignment.